# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [1]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
import os
import shutil

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [2]:
print("Creating temp folder...")
temp_path = "tmp_train"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

print("Copying files...")
shutil.copy(os.path.join("code", "train.py"), temp_path)
shutil.copy(os.path.join("code", "model.py"), temp_path)
shutil.copy(os.path.join("code", "preprocessing.py"), temp_path)
shutil.copy(os.path.join("code", "utils.py"), temp_path)
print("Done.")

Creating temp folder...
Copying files...
Done.


# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [3]:
workspace = Workspace.from_config()
workspace

Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod')

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [4]:
from azureml.core import Experiment
experiment_name = "q3-cnndepthmap-resnet-height-95k"
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
q3-cnndepthmap-resnet-height-95k,cgm-azureml-prod,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [5]:
import os
import urllib
import shutil
import azureml
 
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod'), name=gpu-cluster, id=/subscriptions/9b82ecea-6780-4b85-8acf-d27d79028f07/resourceGroups/cgm-ml-prod/providers/Microsoft.MachineLearningServices/workspaces/cgm-azureml-prod/computes/gpu-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [6]:
dataset_name = "anon-depthmap-95k"
dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('cgm_ml_process', '95k_depthmap_trainingdata/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "de94949f-1022-42bd-bee3-0880c95903f8",
    "name": "anon-depthmap-95k",
    "version": 1,
    "description": "A depthmap based dataset containing 95k artifacts, to be used for training.",
    "workspace": "Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod')"
  }
}

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [7]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0']

In [8]:
# Specify pip packages here.
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2",
    "opencv-python",
    "matplotlib",
    #"tensorflow==2.1.0"
]

script_params = {
    "--split_seed": 666,
    "--target_size": "180x240",
    "--epochs": 1000,
    "--batch_size": 64,
    "--normalization_value": 7.5,
    "--res_blocks": "1,1,1,1",
    "--dropouts": "0.2,0.3,0.4,0.5",
    "--comment": "ResNet on 95K data. With dropout."
}

# Create the estimator.
estimator = TensorFlow(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="train.py",
    use_gpu=True,
    framework_version="2.0",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages,
    script_params=script_params
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator)

# Show outputs.
run

Experiment,Id,Type,Status,Details Page,Docs Page
q3-cnndepthmap-resnet-height-95k,q3-cnndepthmap-resnet-height-95k_1598005774_1e8bf3d0,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [9]:
shutil.rmtree(temp_path)

In [10]:
!pwd

/Users/tristanbehrens/Development/DeepLearning/currentprojects/WHH/cgm-ml-service/Models/CNNDepthMap/CNNDepthMap-height/q3-cnndepthmap-resnet-height


## 